In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS
import plotly
import cufflinks
import warnings
#nltk.download("stopwords")
plt.style.use('ggplot')
warnings.filterwarnings('ignore')
%matplotlib inline

# 2020 Kaggle Data Science Survey Analysis

The 2020 Kaggle Data Science and Machine Learning Survey was conducted by Kaggle to get a comprehensive view of the state of data science and machine learning adoption and usage. The survey was live for 3.5 weeks in October 2020 and collected 20,036 responses.  

The dataset was used to explore reponses from Data Scientists to understand the programming languages, BI tools, preferred IDE's and platforms used. Also, on common topics, libraries and tools used for machine learning, and how they publicly share their work and resources used to gain knowledge about current trends.  

This information can provide insights for both aspiring and experienced data scientists. Some of the questions that can be answered are as follows:

1. What are the popular programming languages, databases, BI tools and IDE's used in this field?
2. What are some common machine learning algorithms and frameworks to be aware of?
3. What are the platforms used to learn and share data science work?
4. What are the tools and frameworks that can be useful to know in that coming future? 

Note: These insights are based on the survey responses and limited only to the questions asked in the survey. There may be other topics which are integral in Data Science.

Only respondents whose current role is a Data Scientist are considered for the following analysis.

In [ ]:
data = pd.read_csv("/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv")
cols = list(data.loc[0])
df = data.iloc[1:].reset_index(drop = True)
df.columns = cols

#Change column names for demograhic information
old_names = list(df.columns[:7])
new_names = ['Duration', 'Age','Gender','Country', 'HighestEducation','CurrentRole', 'YrsCodingExperience']
df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

#assign stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('following')
tokenizer = RegexpTokenizer(r'\w+')

#function to get main topic words from column names
def maintopicwords(cols_list):
    cols = [] 
    for m in cols_list:
        mrepl = m.replace("Select all that apply", "").replace("- Selected Choice","").replace("next 2 years","")
        question = mrepl.split("-", maxsplit = 1)[0].lower()
        choice = mrepl.split("-", maxsplit = 1)[1].strip()
        tokens = tokenizer.tokenize(question)
        qparts = [t for t in tokens if t not in stopwords]
        p1 = " ".join(qparts)
        cols.append(p1+' - '+choice)
    return cols

#change to new column names and filter for only Data Scientists in the Current Role
old_cols = list(df.loc[:,df.nunique() == 1].columns)
new_cols = maintopicwords(old_cols)
df.rename(columns = dict(zip(old_cols, new_cols)), inplace = True)
df.drop(columns = list(df.columns[(df.columns.str.contains('None'))|(df.columns.str.contains('Other'))]), inplace = True)
df_ds = df[(df['CurrentRole'] == 'Data Scientist')&(df['YrsCodingExperience'] != 'I have never written code')]

## Demographics of Data Scientists in the Survey

Understanding the demographics of the respondents help in perceiving the responses in context. 

### Country they reside in

In [ ]:
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot,plot
init_notebook_mode(connected=True)

#data for country information
ds_map = df_ds.Country.value_counts().reset_index().rename(columns = {'Country':'Count', 'index':'Country'})
Total = sum(ds_map['Count'])
ds_map['Pct_Count'] = '%Count = '+(((ds_map['Count']/Total)*100).round(1)).astype('str')+'%'

#data and aesthetics for map
data = dict(
        type = 'choropleth',
        colorscale = 'Viridis_r',
        locations = ds_map['Country'],
        locationmode = "country names",
        z = ds_map['Count'],
        text = ds_map['Pct_Count'],
        colorbar = {'title' : 'Number of Respondents'},
      )
layout = dict(title = 'Country of Respondents ',
              geo = dict(projection = {'type':'mercator'})
             )

There are respondents from 55 countries. Top countries are India (22%) and US (15%), followed by Brazil (5%), UK (4%) and Russia (3%).
The interactive map below shows the number and percentage of respondents from countries around the world. 

In [ ]:
choromap = go.Figure(data = [data],layout = layout)
choromap.update_layout(height = 650, width = 900, margin = {'l':5, 'r':5})
iplot(choromap,validate=False)

India and US together constitute ~36% of respondents, the results can be influenced by the adoptions and trends followed in these two countries. 

### Education

The highest education completed is shown below. The others category consists of respondents who have some/no formal education past high school and those who prefer not answer.

In [ ]:
ed_sel = (df_ds.HighestEducation == 'Master’s degree') | (df_ds.HighestEducation == 'Doctoral degree') | (df_ds.HighestEducation == 'Bachelor’s degree') | (df_ds.HighestEducation == 'Professional degree')
df_ds.loc[~ed_sel, 'HighestEducation'] = "Others"
#plot
ax = df_ds.HighestEducation.value_counts().plot(kind = 'bar', figsize = (10,8))
ax.set_alpha(0.8)
ax.set_title("Education of Data Scientists in the Survey", fontsize=18)
ax.set_ylabel("Number of Respondents", fontsize=14, color = '0.2')
ax.set_xlabel("Highest Education",  fontsize=14, color = '0.2')
ax.set_xticklabels(fontdict = {'fontsize':11},color = '0.27',labels = ['Master’s degree','Bachelor’s degree','Doctoral degree','Others','Professional degree'], rotation = 0)

totals = []

for i in ax.patches:
    totals.append(i.get_height())

total = sum(totals)

for i in ax.patches:
    ax.text(i.get_x()+.07, i.get_height()+2, \
         str(round((i.get_height()/total)*100, 2))+'%', fontsize=12, color='0.25')

68.28% of data scientists have a Master's degree or above. This also means that there are ~30% with bachelor's or lesser degree who are data scientists. Considering the abundance of online learning resources, a Master's degree or above may not be essential in the future.

### Years of Coding Experience

In [ ]:
codingexp_order = ['< 1 years','1-2 years',  '3-5 years','5-10 years', '10-20 years', '20+ years']
ax = df_ds.YrsCodingExperience.value_counts(normalize = True).loc[codingexp_order].plot(kind = 'bar', figsize = (14,8), color = 'seagreen')
ax.set_alpha(0.8)
ax.set_title("Coding Experience of Data Scientists", fontsize=18)
ax.set_ylabel("Percentage of Respondents", fontsize=14, color = '0.2')
ax.set_xlabel("Years of Coding Experience",  fontsize=14, color = '0.2')
ax.set_xticklabels(fontdict = {'fontsize':11},color = '0.27',labels = codingexp_order, rotation = 0)

for i in ax.patches:
    ax.text(i.get_x()+0.1, i.get_height()+0.001, \
            str(round((i.get_height())*100, 2))+'%', fontsize=12, color='0.25')


Coding experience or knowing how to code is important in this field. Majority of the data scientists have more than 2 years of coding experience.

### Age and Gender

In [ ]:
age_gender = df_ds[(df_ds.Gender == 'Man') | (df_ds.Gender == 'Woman')].groupby(['Age', 'Gender']).size().reset_index()
df_ag = pd.pivot(age_gender, index = 'Age', columns = 'Gender').reset_index()
df_ag.columns = ['Age','Male', 'Female']
#plot
ax = df_ag[['Age','Female','Male']].set_index('Age').plot(stacked=True, kind='bar', figsize=(12, 8), rot='horizontal')

for i in ax.patches:
    height = i.get_height()#height of the bar
    width = i.get_width()#width of the bar
    x = i.get_x()#starting (left) x position of bar in the plot 
    y = i.get_y()#starting (bottom) y position of bar in the plot
    
    label_text = int(height) 
    
    label_x = x + width / 2
    label_y = y + height / 2

    if height > 5:
        ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=10, color = 'ivory')
    
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')    
ax.set_ylabel("Number of Respondents", fontsize=16)
ax.set_xlabel("Age Group (in years)", fontsize=16)
ax.set_title("Age and Gender of Data Scientists in the Survey", fontsize = 18, color = '0.2')
plt.show()

82% are male, 16.4% are female and 1.6% belong to other gender descriptions. Majority of the data scientists are males.  

Dominating age group is 25-34 years comprising 45% of the respondents, followed by 22-24 years and 35-39 years with 14% each. This can include people in different stages of their career.

## Technical Skills of Data Scientists

The following are insights on the technical skills of data scientists. This includes top programming languages, tools and platforms used regularly for their work. Also, provides information on topics they would like to learn or explore in the near future. 

### Programming languages, Integrated Development Environments (IDE) and visualization libraries 

In [ ]:
def convert_to_single_list(series):
    text =[]
    for val in series:
        if len(val)> 1:
            text.extend(val)
    return text

def dataframe_to_wordcloudplot(dfw, bg_color, figure_size, title):
    dfwc = dfw.apply(lambda x: x.str.replace(r" ", ""))
    series = dfwc.apply(lambda x: list(x.dropna()), axis = 1).reset_index(drop = True)
    text = convert_to_single_list(series)
    freq = dict(nltk.FreqDist(text))
    wordcloud = WordCloud(max_words = 100,background_color = bg_color).generate_from_frequencies(freq)
    fig = plt.figure(figsize = figure_size)
    plt.title(title)
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()   

In [ ]:
plang_cols = list(df_ds.columns[(df_ds.columns.str.contains('programming languages use regular basis'))|(df_ds.columns.str.contains('data visualization libraries tools use regular basis'))|(df_ds.columns.str.contains('integrated development environments ide use regular basis'))])
df_plang = df_ds[plang_cols].apply(lambda x: x.str.replace(r"\(.*\)", ""))
wc_lang = dataframe_to_wordcloudplot(df_plang, bg_color = "azure", figure_size = (7,10), title = "Programming Languages, IDE's and Visualization Libraries")

Python, SQL and R are top languages for Data Science. Jupyter (notebooks, lab), Visual Studio Code and PyCharm are the common IDE's used.  

Matplotlib, Seaborn and Plotly are popular data visualization, libraries all of which are in Python. Python is the most suggested language for aspiring data scientists among the three.

### Databases, Business Intelligence Tools and Cloud Platforms 

In [ ]:
clbidb_cols = list(df_ds.columns[(df_ds.columns.str.contains("cloud computing platforms use regular basis"))|(df_ds.columns.str.contains("business intelligence tools use regular basis"))|(df_ds.columns.str.contains("big data products relational databases data warehouses data lakes similar use regular basis"))])
df_clbidb = df_ds[clbidb_cols].apply(lambda x: x.str.replace(r"\(.*\)", ""))
wc_clbidb = dataframe_to_wordcloudplot(df_clbidb, bg_color = "honeydew", figure_size = (10,5), title = 'Databases, BI tools and Cloud platforms')

#### Top 3 Tools and Platforms used are
- Databases - **MySQL, PostgresSQL and Microsoft SQL Server**
- Business Intelligence Tools - **Tableau, Microsoft Power BI and Google Data Studio**
- Cloud Computing Platforms - **Amazon Web Services, Google Cloud Platform and Microsoft Azure**

### Popular Machine Learning Algorithms and Frameworks

In [ ]:
ml_cols = df_ds.columns[(((df_ds.columns.str.contains('machine learning'))|(df_ds.columns.str.contains('ml')))&(df_ds.columns.str.contains('regular basis')))|(df_ds.columns.str.contains('use tools help manage machine learning experiments'))]
df_ml = df_ds[ml_cols].apply(lambda x: x.str.replace(r" or ","/")).apply(lambda x: x.str.replace(r"\(.*\)", ""))
wc_clbidb = dataframe_to_wordcloudplot(df_ml, bg_color = "black", figure_size = (10,8), title = 'Machine Learning Algorithms and Frameworks')

#### Commonly used machine learning algorithms:
- Linear/Logistic Regression
- Decision Trees/Random Forests
- Gradient Boosting Machines (Ex: LightGBM, XgBoost)
- Convolutional Neural Networks
<br>
  
#### Popular machine learning frameworks:
- Scikit-learn
- Keras
- Tensor Flow
- XgBoost  


#### Automated machine learning tools are preferred mostly for *Hyperparameter Tuning*, *Model Selection* and *Feature Engineering*. Some of the tools used for this are *Auto-Sklearn*, *Auto-Keras* and *Auto-ml*.


### Resources used to learn and publicly share work

In [ ]:
shln_cols = df_ds.columns[(df_ds.columns.str.contains('favorite media sources report data science topics'))|(df_ds.columns.str.contains('platforms begun completed data science courses'))|(df_ds.columns.str.contains('publicly share deploy data analysis machine learning applications'))]
df_shln = df_ds[shln_cols].apply(lambda x: x.str.replace(r"\(.*\)", ""))
df_shln.drop(columns = list(df_ds.columns[df_ds.columns.str.contains("i do not share my work publicly")]), inplace = True)
wc_shln = dataframe_to_wordcloudplot(df_shln, bg_color = "aliceblue", figure_size = (7,6), title = 'Resources/Platforms to learn and share Data Science work')

Coursera is the most popular for Data Sciences courses, followed by Udemy and University Courses. Favorite media sources are Kaggle, Blogs and Youtube videos. 

Platforms like GitHub, Kaggle and Colab are used to publicly share data science and machine learning work.


### Technical skills Data Scientists hope to learn in the next two years

In [ ]:
befa_cols = df_ds.columns[df_ds.columns.str.contains("hope become familiar")]
df_befa = df_ds[befa_cols].apply(lambda x: x.str.replace(r"\(.*\)", ""))
wc_befa = dataframe_to_wordcloudplot(df_befa, bg_color = "black", figure_size = (10,6), title = 'Machine Learning Frameworks, Cloud Platforms, Databases and other Tools')

### Data Scientists are gearing towards learning more about:
- **Cloud Computing Products/Services related to Data Science, Machine Learning and AI offered by**
    - Amazon Web Services, Google Cloud Platform, Microsoft Azure  
  <br>
- **Automated Machine Learning Tools for**
    - Automated Model Selection (Auto-Sklearn)
    - Automation of Full ML Pipelines (Auto-Keras)
    - Automated Feature Engineering and Hyperparameter Tuning    
<br>
- **Business Intelligence Tools such as**
    - Tableau
    - Microsoft Power BI
    - Google Data Studio  
    
The trend is moving towards understanding relevant cloud computing services as many industries are adopting cloud infrastructure for their data. Also, full/partial automation of the machine learning pipeline has gaining momentum.
If you have just started working as a data scientist, knowledge of these topics and tools can keep you up to date (valuable and relevant) in the coming future.